In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import geopandas as gpd
import random
from scipy.stats import truncnorm, norm
from collections import defaultdict
import math
from shapely.ops import transform
from pyproj import Geod
import pyarrow
from datetime import datetime
import sys
import importlib
from dataclasses import dataclass
import yaml
from typing import Union

In [4]:
# добавляем папку modules в sys.path, чтобы импортировать функции созданные ранее

# sys.path.append(os.path.abspath(os.path.join("..", "modules")))

In [501]:
# Функции генерации паттернов времени и базовые функции времени созданные ранее
import data_generator.fraud.time
import data_generator.fraud.txndata
import data_generator.configs
import data_generator.utils
importlib.reload(data_generator.fraud.time)
importlib.reload(data_generator.fraud.txndata)
importlib.reload(data_generator.configs)
importlib.reload(data_generator.utils)
from data_generator.general_time import *
from data_generator.utils import build_transaction, create_txns_df, sample_category, get_values_from_truncnorm
from data_generator.fraud.txndata import FraudTxnPartData, TransAmount
from data_generator.fraud.time import derive_from_last_time
from data_generator.configs import PurchBehaviorHandler

In [10]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [7]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [8]:
os.chdir("..")

In [9]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [18]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

# Доп. данные для генерации фрод транзакций

**Загрузка данных:**
- оффлайн мерчантов
- онлайн мерчантов
- девайсов клиентов и мошенников
- городов с полигонами
- городов с координатами центров
- клиентов
- фрод IP адресов
- категорий и их характеристик
- категорий для дропов покупателей
- Счетов клиентов
- Внешних счетов

In [612]:
offline_merchants = gpd.read_file("./data/cleaned_data/offline_merchants_points.gpkg")
online_merchant_ids = pd.read_csv("./data/cleaned_data/online_merchant_ids.csv").iloc[:, 0] # нужны в виде серии
client_devices = pd.read_csv("./data/cleaned_data/client_devices.csv")
fraud_devices = pd.read_csv("./data/cleaned_data/fraud_devices.csv")
districts_ru = gpd.read_file("./data/cleaned_data/district_ru.gpkg")
area_centers = gpd.read_file("./data/cleaned_data/area_centers.gpkg")
clients_with_geo = gpd.read_file("./data/cleaned_data/clients_with_geo.gpkg") 
fraud_ips = gpd.read_file("./data/cleaned_data/fraud_ips.gpkg")
cat_stats_full = pd.read_csv("./data/cleaned_data/cat_stats_full.csv")
cat_fraud_amts = pd.read_csv("./data/cleaned_data/cat_fraud_amts.csv")
drop_purch_cats = pd.read_csv("./data/cleaned_data/drop_purch_cats.csv")
accounts = pd.read_csv("./data/generated_data/accounts.csv")
outer_accounts = pd.read_csv("./data/generated_data/outer_accounts.csv").iloc[:, 0] # нужны в виде серии

**Датафрейм под транзакции. Этап тестов**<br>
Создадим пустой датафрейм под транзакции. Будем копировать и использовать его при тестах функций.

In [13]:
transactions = pd.DataFrame(
            {"client_id": pd.Series(dtype="int64"),
            "txn_time": pd.Series(dtype="datetime64[ns]"),
             "unix_time":pd.Series(dtype="int64"),
            "amount": pd.Series(dtype="float64"),
            "type": pd.Series(dtype="string"),
            "channel": pd.Series(dtype="string"),
            "category": pd.Series(dtype="string"),
            "online":pd.Series(dtype="bool"),
            "merchant_id":pd.Series(dtype="int64"),
             "trans_city":pd.Series(dtype="string"),
            "trans_lat":pd.Series(dtype="float64"),
             "trans_lon":pd.Series(dtype="float64"),
            "trans_ip":pd.Series(dtype="string"),
             "device_id":pd.Series(dtype="int64"),
             "account": pd.Series(dtype="int64"),
            "is_fraud": pd.Series(dtype="bool"),
             "is_suspicious": pd.Series(dtype="bool"),
            "status":pd.Series(dtype="string"),
            "rule":pd.Series(dtype="string")})
transactions

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule


**Функция получения случайных значений из `truncnorm` распределения**
- пойдет в модуль `utils`

In [14]:
def get_values_from_truncnorm(low_bound, high_bound, mean, std, size=1):
    """
    Сгенерировать массив чисел из обрезанного нормального распределения.
    Можно сгенерировать массив с одним числом
    ------------
    low_bound - float, int. Нижняя граница значений
    high_bound - float, int. Верхняя граница значений 
    mean - float, int. Среднее
    std - float, int. Стандартное отклонение
    size - Количество чисел в возвращаемом массиве
    ------------
    Возвращает np.ndarray
    """
    return truncnorm.rvs((low_bound - mean) / std, (high_bound - mean) / std, loc=mean, scale=std, size=size)

## Класс `DropDistributorCfg`
- конфигурации на основании которых будут генерироваться транзакции дропов
  занятых распределением полученных денег: переводами, снятиями, покупкой криптовалюты.

In [52]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

### Создание аргументов для `DropDistributorCfg`

In [16]:
# timestamp-ы использованные еще для генерации легальных и purchase фрод транзакций
timestamps = pd.read_parquet("./data/generated_data/timestamps.parquet", engine="pyarrow")

In [19]:
time_weight_args = time_cfg["time_weights_args"]

In [20]:
time_weights_dict = get_all_time_patterns(time_weight_args)

**Легальные транзакции**  
Нужно знать их количество для определения числа фрод транзакций  
и следовательно, количества клиентов для их генерации

In [21]:
legit_txns = pd.read_parquet("./data/generated_data/legit_trans_148K.parquet", engine="pyarrow")

**Клиенты для дроп фрода**  
Случайно возьмем клиентов, у которых нет никаких транзакций  
Для этого загрузим семплированных для легальных транзакций клиентов, т.е. у кого были созданы легальные транзакции.  
У части этих клиентов также созданы purchase фрод транзакции. Всех этих клиентов надо исключить.

In [22]:
clients_sample_df = gpd.read_file("./data/cleaned_data/clients_sample.gpkg")

In [23]:
clients_sample_df.shape

(3000, 12)

Конфиги загружаются временно тут

In [316]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [24]:
# доля дроп фрода - Дропы занимающиеся переводами и снятиями; и дропы, которые покупают товары на присланные деньги
fraud_rate = fraud_cfg["fraud_rates"]["total"]
drop_dist_share = drops_cfg["distributor"]["rate"] # Дропы распределители. Доля от всего фрода
drop_purch_share = drops_cfg["purchaser"]["rate"] # Дропы покупатели. Доля от всего фрода

In [25]:
# лимит на исходящие транзакции дропа. Нужен сейчас для вычисления количества дропов распределителей

dist_out_lim = drops_cfg["distributor"]["out_lim"]

In [33]:
# отсюда посчитаем количество клиентов для дроп фрода с распределением денег

legit_count = legit_txns.shape[0]
# подсчет количества транзакций равных 1% от всех транзакций
# т.к. не все транзакции еще созданные, то считаем основываясь на количестве легальных транзакций и fraud rate
one_perc = round(legit_count / ((1 - fraud_rate) * 100))
# Один процент транзакций умножаем на долю транзакций дропов-распределителей и делим на максимальное количество
# исходящих транзакций которое распределитель может сделать до детекта. Так находим сколько примерно дропов будет под такой фрод
dist_drops_count = round(one_perc * drop_dist_share / dist_out_lim) 
dist_drops_count

28

In [34]:
# Клиенты незадействованные ранее

not_used_clients = clients_with_geo.loc[~clients_with_geo.client_id.isin(clients_sample_df.client_id)].copy()

In [35]:
# семплируем dist_drops_count клиентов

dist_drop_clients = not_used_clients.sample(n=dist_drops_count).reset_index(drop=True)

In [36]:
dist_drop_clients.head(2)

,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,52,32,1924-12-12,male,Курская,Курск,UTC+3,51.730339,36.192645,414595,2.60.0.50,"MULTIPOLYGON (((36.0603 51.67692, 36.06039 51...."
1,403,42,1973-05-26,female,Астраханская,Астрахань,UTC+4,46.365581,48.055998,520662,2.60.1.127,"MULTIPOLYGON (((47.87112 46.26966, 47.87127 46..."


In [37]:
dist_drop_clients.shape

(28, 12)

## Объект датакласса `DropDistributorCfg` конфиги для дропов распределителей

In [1015]:
# Импорт на время разработки
import data_generator.configs
importlib.reload(data_generator.configs)
from data_generator.configs import DropDistributorCfg

**Доп. конфиги из `drop.yaml`**

In [1016]:
# временно

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [1017]:
# Параметры обозначенные в config/drops.yaml

dist_cfg = drops_cfg["distributor"]
dist_in_lim = dist_cfg["in_lim"]
dist_out_im = dist_cfg["out_lim"]
period_in_lim = dist_cfg["period_in_lim"]
period_out_lim = dist_cfg["period_out_lim"]
dist_lag_int = drops_cfg["lag_interval"]
two_way_delta = drops_cfg["two_way_delta"]
pos_delta = drops_cfg["pos_delta"]
split_rate = dist_cfg["split_rate"]
chunks = dist_cfg["chunks"]
dist_inbound = drops_cfg["inbound_amt"]
dist_round = dist_cfg["round"]
dist_trf_max = dist_cfg["trf_max"]
dist_reduce = dist_cfg["reduce_share"]
dist_att = dist_cfg["attempts"]
to_drops = dist_cfg["to_drops"]
crypto_rate = dist_cfg["crypto_rate"]

In [1018]:
dist_configs = DropDistributorCfg(clients=dist_drop_clients, timestamps=timestamps, accounts=accounts, \
                                  outer_accounts=outer_accounts, client_devices=client_devices, atms=None, \
                                  online_merchant_ids=online_merchant_ids, cities=districts_ru, in_lim=dist_in_lim, 
                                  out_lim=dist_out_im, period_in_lim=period_in_lim, period_out_lim=period_out_lim, \
                                  lag_interval=dist_lag_int, two_way_delta=two_way_delta, pos_delta=pos_delta, \
                                  split_rate=split_rate, chunks=chunks, inbound_amt=dist_inbound, round=dist_round, \
                                  trf_max=dist_trf_max, reduce_share=dist_reduce, attempts=dist_att, to_drops=to_drops, \
                                  crypto_rate=crypto_rate
                                 )

## Класс `DropPurchaserCfg` конфиги для дропов покупателей

In [237]:
@dataclass
class DropPurchaserCfg: # <-------------------- in development. Совсем не откорректирован.
    """
    Это данные на основе которых будут генерироваться транзакции дропов-покупателей
    ---------------------
    clients: pd.DataFrame
    timestamps: pd.DataFrame
    transactions: pd.DataFrame
    accounts: pd.DataFrame. Номера счетов клиентов. 
    outer_accounts: pd.Series. Номера внешних счетов для транзакций вне банка.
    client_devices: pd.DataFrame
    offline_merchants: pd.DataFrame
    categories: pd.DataFrame
    online_merchant_ids: pd.Series
    time_weights_dict: dict
    rules: pd.DataFrame
    cities: pd.DataFrame
    fraud_amounts: pd.DataFrame
    period_in_lim: int. Количество входящих транзакций после которых дроп уходит на паузу.
    period_out_lim: int. Количество исходящих транзакций после которых дроп уходит на паузу.
    lag_interval: int. Минуты. На сколько дроп должен делать паузу после
                       достижения лимита входящих и/или исходящих транзакций.
                       Например 1440 минут(сутки). Отсчет идет от первой транзакции в последнем периоде активности.
    two_way_delta: dict. Минимум и максимум дельты времени. Для случаев когда дельта может быть и положительной и отрицательной.
                         Эта дельта прибавляется к lag_interval, чтобы рандомизировать время возобновления активности,
                         чтобы оно не было ровным. Берется из конфига drops.yaml
    pos_delta: dict. Минимум и максимум случайной дельты времени в минутах. Для случаев когда дельта может быть только положительной.
                          Эта дельта - промежуток между транзакциями дропа в одном периоде. Просто прибавляется ко времени последней транзакции.
    chunks: dict. Характеристики для генератора сумм транзакций по частям.
    inbound_amt: dict. Настройки для сумм входящих транзакций
    round: int. Округление целой части сумм транзакций. Напр. 500 значит что суммы будут кратны 500 - кончаться на 500 или 000
    """
    clients: pd.DataFrame
    timestamps: pd.DataFrame
    transactions: pd.DataFrame
    accounts: pd.DataFrame
    outer_accounts: pd.Series
    client_devices: pd.DataFrame
    offline_merchants: pd.DataFrame
    categories: pd.DataFrame
    online_merchant_ids: pd.Series
    time_weights_dict: dict
    rules: pd.DataFrame
    cities: pd.DataFrame
    fraud_amounts: pd.DataFrame
    period_in_lim: int
    period_out_lim: int
    lag_interval: int
    two_way_delta: dict
    pos_delta: dict
    chunks: dict
    inbound_amt: dict
    round: dict

## Класс `DropTxnPartData` - генерация части данных транзакции
-  локация, мерчант id, ip адрес, device id, канал, тип транзакции. 

In [ ]:
class DropTxnPartData:
    """
    Класс для генерации части данных о транзакции дропа:
    канал, тип операции, мерчант, геопозиция, город, IP адрес, иногда статус.
    ------------------
    Атрибуты:
    --------
    client_info - pd.DataFrame или namedtuple. Запись с информацией о клиенте
    online_merchant_ids- pd.Series. id онлайн мерчантов
    client_devices - pd.DataFrame. Девайсы клиентов.
    last_txn - tuple. Для кэширования данных любой последней транзакции.
    """
    def __init__(self, configs: Union[DropDistributorCfg, DropPurchaserCfg]):
        """
        configs: Один из датаклассов — DropDistributorCfg или DropPurchaserCfg.
                 Содержит параметры и конфиги для генерации фрода.
        """
        self.client_info = None
        self.online_merchant_ids = configs.online_merchant_ids
        self.client_devices = configs.client_devices
        self.last_txn = None


    def assert_client_info(self):
        """
        Проверка что self.client_info не пустое
        """
        assert self.client_info is not None, \
            f"self.client_info is {type(self.client_info)}"


    def original_purchase(self, online=True, get_cached=False):
        """
        Оригинальные данные клиента для операций покупок.
        На данный момент это для дропов.
        Для операций на криптобирже и для покупки товаров дропами
        -------
        online: bool.
        get_cached: bool. Пробовать ли вернуть последние кэшированные данные
                    вместо генерации новых.
        """
        self.assert_client_info()

        if get_cached and self.last_txn is not None:
            return self.last_txn

        if online:
            merchant_id = self.online_merchant_ids.sample(n=1).iat[0]
            # Координаты города и название
            trans_lat = self.client_info.lat
            trans_lon = self.client_info.lon
            trans_ip = self.client_info.home_ip
            trans_city = self.client_info.area        
            # Семпл девайса клиента
            devices = self.client_devices.loc[self.client_devices.client_id == self.client_info.client_id]
            device_id = devices.device_id.sample(1).iloc[0]
            txn_type = "purchase"
            # Не генерируем channel. Он должен быть определен вовне
            channel = None

            self.last_txn = merchant_id, trans_lat, trans_lon, trans_ip, trans_city, \
                            device_id, channel, txn_type
            self.last_txn = self.last_txn
            return self.last_txn

        
    def original_data(self, online, receive=None):
        """
        Получение оригинальных данных клиента для транзакции.
        Пока этот метод для клиентов дропов и, возможно, для переводов мошенникам
        ------------------------------------
        online - bool.
        received - bool.
        """
        self.assert_client_info()

        # Входящий перевод
        if online and receive:
            trans_ip = "not applicable"
            device_id = np.nan
            channel = "transfer"
            txn_type = "inbound"
            merchant_id = np.nan
            trans_lat = np.nan
            trans_lon = np.nan
            trans_city = "not applicable"

            self.last_txn = merchant_id, trans_lat, trans_lon, trans_ip, trans_city, \
                            device_id, channel, txn_type
            return self.last_txn
        
        client_info = self.client_info
        client_devices = self.client_devices
        # Исходящий перевод
        if online:
            # Для онлайна просто берется home_ip и device_id из данных клиента.
            trans_ip = client_info.home_ip
            devices = client_devices.loc[client_devices.client_id == client_info.client_id]
            device_id = devices.device_id.sample(1).iloc[0]
            channel = "transfer"
            txn_type = "outbound"  

        # Оффлайн. Снятие в банкомате
        elif not online:
            trans_ip = "not applicable"
            device_id = np.nan
            channel = "ATM"
            txn_type = "withdrawal"
            
        merchant_id = np.nan
        # Локация транзакции просто записываем координаты и название города клиента
        trans_lat = client_info.lat
        trans_lon = client_info.lon
        trans_city = client_info.area

        self.last_txn = merchant_id, trans_lat, trans_lon, trans_ip, trans_city, \
                        device_id, channel, txn_type
        return self.last_txn
    
    
    def check_previous(self, dist, last_full):
        """
        Решить можно ли вернуть данные последней транзакции
        из кэша.
        ------
        dist: bool. Дроп распределитель или нет.
        last_full: dict. Полные данные последней транзакции.
                   Нужны, чтобы узнать channel.
        """
        if not dist:
            return False
        if last_full is None:
            return False
        if last_full["channel"] == "crypto_exchange":
            return True
        
        return False

    
    def reset_cache(self):
        """
        Сброс кэша
        """
        self.last_txn = None

In [853]:
# Временный импорт

import data_generator.fraud.txndata
importlib.reload(data_generator.fraud.txndata)
from data_generator.fraud.txndata import DropTxnPartData

txn_part_data_test = DropTxnPartData(configs=dist_configs)

for client in dist_drop_clients.itertuples():
    txn_part_data_test.client_info = client
txn_part_data_test.client_info

Pandas(Index=27, client_id=484, district_id=28, birth_date='1959-09-13', sex='female', region='Челябинская', area='Магнитогорск', timezone='UTC+5', lat=53.4071891, lon=58.9791432, population=408401, home_ip='2.60.1.205', geometry=<MULTIPOLYGON (((58.897 53.472, 58.899 53.472, 58.901 53.472, 58.903 53.472,...>)

In [854]:
txn_part_data_test.last_txn

In [855]:
get_cached = txn_part_data_test.check_previous(dist=True, last_full=None)
txn_part_data_test.original_purchase(online=True, get_cached=get_cached)

(np.int64(6822),
 53.4071891,
 58.9791432,
 '2.60.1.205',
 'Магнитогорск',
 np.int64(839),
 None,
 'purchase')

In [856]:
last_full = {"channel":"crypto_exchange"}
get_cached = txn_part_data_test.check_previous(dist=True, last_full=last_full)
txn_part_data_test.original_purchase(online=True, get_cached=get_cached)
# merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, txn_type

(np.int64(6822),
 53.4071891,
 58.9791432,
 '2.60.1.205',
 'Магнитогорск',
 np.int64(839),
 None,
 'purchase')

In [857]:
last_full = {"channel":"crypto_exchange"}
get_cached = txn_part_data_test.check_previous(dist=False, last_full=last_full)
txn_part_data_test.original_purchase(online=True, get_cached=get_cached)

(np.int64(6853),
 53.4071891,
 58.9791432,
 '2.60.1.205',
 'Магнитогорск',
 np.int64(839),
 None,
 'purchase')

In [488]:
txn_part_data_test.original_data(online=False)

(nan,
 53.4071891,
 58.9791432,
 'not applicable',
 'Магнитогорск',
 nan,
 'ATM',
 'withdrawal')

In [489]:
txn_part_data_test.last_txn

(nan,
 53.4071891,
 58.9791432,
 'not applicable',
 'Магнитогорск',
 nan,
 'ATM',
 'withdrawal')

In [490]:
txn_part_data_test.reset_cache()
txn_part_data_test.last_txn

## Объект класса `DropTxnPartData` для распределителей
- генерация части данных транзакции

In [92]:
part_txn_data_dist = DropTxnPartData(configs=dist_configs)

## Класс `DropBaseClasses`  
Агрегатор базовых классов для дропов

In [ ]:
class DropBaseClasses:
    """
    acc_hand: DropAccountHandler. Управление счетами транзакций.
    amt_hand: DropAmountHandler. Управление суммами транзакций.
    part_data: DropTxnPartData. Генерация части данных транзакции:
               гео, ip, город, мерчант id и т.п.
    time_hand: DropTimeHandler. Генерация времени транзакций.
    behav_hand: DistBehaviorHandler| PurchBehaviorHandler. Управление поведением дропа
    """
    acc_hand: DropAccountHandler
    amt_hand: DropAmountHandler
    part_data: DropTxnPartData
    time_hand: DropTimeHandler
    behav_hand: Union[DistBehaviorHandler, PurchBehaviorHandler]

## Класс `CreateDropTxn`

In [270]:
class CreateDistTxn:
    """
    Создание транзакций дропа распределителя под разное поведение.
    """
    def __init__(self, timestamps, trans_partial_data, drop_client_cls, in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, \
                attempts=0):
        """
        timestamps - pd.DataFrame.
        trans_partial_data - FraudTransPartialData. Генератор части данных транзакции - мерчант, гео, ip, девайс и т.п.
        drop_client_cls - DropClient. Генератор активности дропов: суммы, счета, баланс
        in_txns - int. Количество входящих транзакций
        out_txns - int. Количество исходящих транзакций
        in_lim - int. Лимит входящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        out_lim - int. Лимит исходящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        last_txn - dict. Полные данные последней транзакции
        attempts - int. Сколько попыток совершить операцию будет сделано дропом после первой отклоненной транзакции.
        """
        self.timestamps = timestamps
        self.trans_partial_data = trans_partial_data
        self.drop_client = drop_client_cls
        self.in_txns = in_txns
        self.out_txns = out_txns
        self.in_lim = in_lim
        self.out_lim = out_lim
        self.last_txn = last_txn
        self.attempts = attempts


    def cash_flow_action(self, online, declined, in_chunks, to_drop_share=0.2, receive=False): # <------------------------- KEEP, EDIT
        """
        Один входящий/исходящий перевод либо одно снятие в банкомате.
        ---------------------
        online - bool. Онлайн перевод или снятие в банкомате.
        declined - bool. Будет ли текущая транзакция отклонена.
        in_chunks - bool. Транзакция будет частью серии транзакций.
        to_drop_share - float. Вероятность, что дроп пошлет другому дропу
        receive - входящий перевод или нет.
        """
        client_id = self.trans_partial_data.client_info.client_id # берем из namedtuple
        
        # Время транзакции. Оно должно быть создано до увеличения счетчика self.in_txns
        txn_time, txn_unix = self.get_txn_time(in_lim=2, out_lim=5, lag_interval=1440)

        # перевод дропу
        if receive:
            self.in_txns += 1
            amount = self.drop_client.receive(declined=declined)
            account = self.drop_client.account
        # перевод от дропа    
        elif not receive and online:
            to_drop = np.random.choice([True, False], p=[to_drop_share, 1 - to_drop_share])
            self.out_txns += 1
            account = self.drop_client.get_account(to_drop=to_drop)
        # снятие дропом    
        elif not receive and not online:
            account = self.drop_client.account
            self.out_txns += 1
        
        # Генерация части данных транзакции. Здесь прописываются аргументы online и receive
        merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, type = \
                                                self.trans_partial_data.original_transfer_or_atm(online=online, receive=receive)
        
        # Генерация суммы если исходящая транзакция
        # Если эта транзакция только часть серии операций для распределения всего баланса
        if in_chunks:
            chunk = self.drop_client.get_chunk_size(online=online, atm_min=10000, start=5000, stop=25000, step=5000)
            amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)
            
        # Иначе если не по частям и не входящая транзакция. not receive т.к. этот метод и для входящих транзакций
        # а у входящих транзакций своя генерация суммы
        elif not in_chunks and not receive:
            amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)

        if declined:
            status = "declined"
            is_fraud = True
            rule = "drop_flow_cashout"
        else:
            status = "approved"
            is_fraud = False
            rule = "not applicable"

        # Статичные характеристики
        is_suspicious = False
        category_name="not applicable"

        # Сборка всех данных в транзакцию и запись как послдней транзакции
        self.last_txn = build_transaction(client_id=client_id, txn_time=txn_time, txn_unix=txn_unix, amount=amount, type=type, channel=channel, \
                             category_name=category_name, online=online, merchant_id=merchant_id, trans_city=trans_city, \
                             trans_lat=trans_lat, trans_lon=trans_lon, trans_ip=trans_ip, device_id=device_id, account=account, \
                             is_fraud=is_fraud, is_suspicious=is_suspicious, status=status, rule=rule)

        return self.last_txn

    
    def limit_reached(self): # <------------------------- KEEP
        """
        Проверка достижения лимитов входящих и исходящих транзакций
        Сверка с self.in_lim и self.out_lim
        ------------------------
        Вернет True если какой либо лимит достигнут
        """
        if self.in_lim == self.in_txns:
            return True
        if self.out_lim == self.out_txns:
            return True
        return False

    def reset_cache(self, only_counters=True): # <------------------------- KEEP, EDIT
        """
        Сброос кэшированных данных
        -------------
        only_counters - bool
                        Если True будут сброшены: self.in_txns, self.out_txns, self.attempts.
                        Если False то также сбросится информация о последней транзакции self.last_txn
        """
        
        self.in_txns = 0
        self.out_txns = 0
        self.batch_txns = 0

        if only_counters:
            return

        self.last_txn = {}

**Тест `CreateDropTxn`**

In [992]:
import data_generator.indev
import data_generator.fraud.drops.base
import data_generator.configs
import data_generator.fraud.drops.behavior
import data_generator.fraud.txndata
import data_generator.fraud.drops.time

importlib.reload(data_generator.indev)
importlib.reload(data_generator.fraud.drops.base)
importlib.reload(data_generator.configs)
importlib.reload(data_generator.fraud.drops.behavior)
importlib.reload(data_generator.fraud.txndata)
importlib.reload(data_generator.fraud.drops.time)

from data_generator.fraud.drops.base import DropAccountHandler
from data_generator.fraud.drops.base import DropAmountHandler
from data_generator.fraud.drops.behavior import DistBehaviorHandler
from data_generator.indev import CreateDropTxn
from data_generator.configs import DropBaseClasses
from data_generator.fraud.txndata import DropTxnPartData
from data_generator.fraud.drops.time import DropTimeHandler

acc_hand1 = DropAccountHandler(configs=dist_configs)
amt_hand1 = DropAmountHandler(configs=dist_configs)
part_data1 = DropTxnPartData(configs=dist_configs)
time_hand1 = DropTimeHandler(configs=dist_configs)
behav_hand1 = DistBehaviorHandler(configs=dist_configs, amt_hand=amt_hand1)
base_agg1 = DropBaseClasses(acc_hand=acc_hand1, amt_hand=amt_hand1, part_data=part_data1, \
                            time_hand=time_hand1, behav_hand = behav_hand1)

cr_drop_txn1 = CreateDropTxn(configs=dist_configs, base=base_agg1, \
                 categories=drop_purch_cats)

for client in dist_drop_clients.iloc[[8]].itertuples():
    part_data1.client_info = client
    acc_hand1.client_id = client.client_id
    acc_hand1.get_account(own=True)
part_data1.client_info

Pandas(Index=8, client_id=11794, district_id=31, birth_date='1970-10-07', sex='female', region='Томская', area='Томск', timezone='UTC+7', lat=56.4847036, lon=84.9481737, population=522940, home_ip='2.60.20.87', geometry=<MULTIPOLYGON (((84.828 56.546, 84.828 56.546, 84.828 56.546, 84.828 56.546,...>)

In [895]:
def reset_caches(cr_drop_txn, behav_hand, amt_hand, time_hand, part_data):
    cr_drop_txn.reset_cache()
    behav_hand.reset_cache(all=False)
    amt_hand.reset_cache(life_end=True) # batch_txns здесь
    time_hand.reset_cache()
    part_data.reset_cache()

**`CreateDropTxn.trf_or_atm` тест**

In [719]:
cr_drop_txn1.in_txns, cr_drop_txn1.out_txns, cr_drop_txn1.last_txn, amt_hand1.batch_txns

(0, 0, None, 0)

In [720]:
cr_drop_txn1.txn_part_data.client_info

Pandas(Index=8, client_id=11794, district_id=31, birth_date='1970-10-07', sex='female', region='Томская', area='Томск', timezone='UTC+7', lat=56.4847036, lon=84.9481737, population=522940, home_ip='2.60.20.87', geometry=<MULTIPOLYGON (((84.828 56.546, 84.828 56.546, 84.828 56.546, 84.828 56.546,...>)

**`category_and_channel`**

In [721]:
cr_drop_txn1.category_and_channel(dist=True)

('crypto_exchange', 'balance_top_up')

In [532]:
channels1 = []
categories1 = []
chan_and_cat1 = pd.DataFrame()

for _ in range(1000):
    chan1, cat1 = cr_drop_txn1.category_and_channel(dist=False)
    channels1.append(chan1)
    categories1.append(cat1)
    
chan_and_cat1["channel"] = channels1
chan_and_cat1["category"] = categories1

In [535]:
drop_purch_cats

,category,weight
0,shopping_net,0.60
1,misc_net,0.25
2,travel_net,0.15


In [536]:
chan_and_cat1.value_counts(normalize=True).reset_index()

,channel,category,proportion
0,ecom,shopping_net,0.596
1,ecom,misc_net,0.256
2,ecom,travel_net,0.148


**`status_and_rule`**  
значения: status, is_fraud, rule

In [554]:
cr_drop_txn1.status_and_rule(declined=False, dist=False)

('approved', False, 'not applicable')

### **`trf_or_atm`**

НЕ отклоненный входящий перевод  
`BehavHand.online=None, receive=True, declined=False, BehavHand.in_chunks=False`

In [569]:
acc_hand1.accounts.query("client_id == 11794")["account"]
acc_hand1.accounts.loc[acc_hand1.accounts.client_id == acc_hand1.client_id, "account_id"].iat[0]

,client_id,account_id,is_drop
5206,11794,15206,False


In [993]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.online = None
behav_hand1.in_chunks = None
receive_txn = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)

pd.DataFrame([receive_txn])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-09 23:00:00,1736463600,42500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable


In [723]:
amt_hand1.balance, behav_hand1.batch_txns

(np.float64(36000.0), 0)

In [724]:
cr_drop_txn1.in_txns, cr_drop_txn1.out_txns

(1, 0)

НЕ отклоненный исходящий перевод целиком  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=False`

In [725]:
part_data1.client_info.area

'Томск'

In [994]:
behav_hand1.online = True
behav_hand1.in_chunks = False

whole_out = cr_drop_txn1.trf_or_atm(receive=False, to_drop=False, declined=False)
pd.concat([pd.DataFrame([receive_txn]), pd.DataFrame([whole_out])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-09 23:00:00,1736463600,42500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-09 23:36:00,1736465760,42500.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,17770,False,False,approved,not applicable


НЕ отклоненное снятие целиком  
`BehavHand.online=False, receive=False, declined=False, BehavHand.in_chunks=False, BehavHand.scen = "atm"`

In [995]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm"
behav_hand1.guide_scenario()
behav_hand1.in_chunks = False

receive_txn2 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
whole_atm = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=False)

pd.concat([pd.DataFrame([receive_txn2]), pd.DataFrame([whole_atm])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-10 12:37:00,1736512620,24700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-10 15:02:00,1736521320,24700.0,withdrawal,ATM,not applicable,False,NaN,Томск,56.484704,84.948174,not applicable,NaN,15206,False,False,approved,not applicable


In [920]:
amt_hand1.balance, amt_hand1.batch_txns

(np.float64(0.0), 1)

НЕ отклоненный исходящий перевод частями  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=True, BehavHand.scen = "split_transfer"`

In [996]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks = True

receive_txn3 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)

all_txns3 = [receive_txn3]
while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=False)
    all_txns3.append(part_out)
pd.DataFrame(all_txns3)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-03 15:51:00,1735919460,24600.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-03 18:22:00,1735928520,7000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,20935,False,False,approved,not applicable
2,11794,2025-01-03 19:16:00,1735931760,10000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,20245,False,False,approved,not applicable
3,11794,2025-01-03 20:46:00,1735937160,7000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,16278,False,False,approved,not applicable
4,11794,2025-01-03 21:52:00,1735941120,600.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,20575,False,False,approved,not applicable


In [923]:
amt_hand1.balance, amt_hand1.batch_txns

(np.float64(0.0), 3)

НЕ отклоненное снятие и перевод(ы) частями

In [997]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm+transfer"
behav_hand1.in_chunks = True

receive_txn4 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns4 = [receive_txn4]

while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=False)
    all_txns4.append(part_out)
pd.DataFrame(all_txns4)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-19 00:30:00,1737246600,45700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-19 02:48:00,1737254880,15900.0,withdrawal,ATM,not applicable,False,NaN,Томск,56.484704,84.948174,not applicable,NaN,15206,False,False,approved,not applicable
2,11794,2025-01-19 05:37:00,1737265020,19000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,16611,False,False,approved,not applicable
3,11794,2025-01-19 07:04:00,1737270240,9000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,22249,False,False,approved,not applicable
4,11794,2025-01-19 08:01:00,1737273660,1800.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,16748,False,False,approved,not applicable


In [763]:
amt_hand1.balance, amt_hand1.batch_txns

(np.float64(0.0), 5)

Отклоненный входящий перевод  
`BehavHand.online=None, receive=True, declined=True, BehavHand.in_chunks=False`

In [764]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)

behav_hand1.online = None
behav_hand1.in_chunks = None
receive_txn5 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=True)

pd.DataFrame([receive_txn5])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-05 11:52:00,1736077920,40000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,True,False,declined,drop_flow_cashout


In [766]:
amt_hand1.balance, amt_hand1.batch_txns

(0, 0)

Отклоненный исходящий перевод целиком  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=False`

In [1001]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "transfer"
behav_hand1.in_chunks = False

receive_txn4 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns4 = [receive_txn4]
i = 0
while amt_hand1.balance > 0 and i < 4:
    behav_hand1.guide_scenario()
    part_out = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
    all_txns4.append(part_out)
    i += 1
pd.DataFrame(all_txns4)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-02 15:15:00,1735830900,55500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-02 18:07:00,1735841220,55500.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,16916,True,False,declined,drop_flow_cashout
2,11794,2025-01-02 19:35:00,1735846500,41700.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,19357,True,False,declined,drop_flow_cashout
3,11794,2025-01-02 22:18:00,1735856280,27900.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,19292,True,False,declined,drop_flow_cashout
4,11794,2025-01-02 23:02:00,1735858920,14100.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,22143,True,False,declined,drop_flow_cashout


In [1002]:
amt_hand1.balance, amt_hand1.batch_txns, amt_hand1.declined_txns

(np.float64(55500.0), 4, 4)

**Отклоненное снятие целиком**  
`BehavHand.online=False, receive=False, declined=True, BehavHand.in_chunks=False`

In [1004]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm"
behav_hand1.in_chunks = False

receive_txn7 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)

behav_hand1.guide_scenario()
whole_out7 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
pd.DataFrame([receive_txn7, whole_out7])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-22 22:24:00,1737584640,31600.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-23 00:34:00,1737592440,31600.0,withdrawal,ATM,not applicable,False,NaN,Томск,56.484704,84.948174,not applicable,NaN,15206,True,False,declined,drop_flow_cashout


In [1006]:
amt_hand1.balance, amt_hand1.batch_txns, amt_hand1.declined_txns

(np.float64(31600.0), 1, 1)

**Отклоненный  исходящий перевод частями**  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=True`

In [1007]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks = True

receive_txn8 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns8 = [receive_txn8]
i = 0
while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out8 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
    all_txns8.append(part_out8)
    i += 1
    if i > 4:
        break
        
pd.DataFrame(all_txns8)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-22 23:58:00,1737590280,25300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-23 02:01:00,1737597660,13000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,15830,True,False,declined,drop_flow_cashout
2,11794,2025-01-23 04:03:00,1737604980,9800.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,17257,True,False,declined,drop_flow_cashout
3,11794,2025-01-23 06:54:00,1737615240,6600.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,15744,True,False,declined,drop_flow_cashout
4,11794,2025-01-23 07:25:00,1737617100,3400.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,17677,True,False,declined,drop_flow_cashout
5,11794,2025-01-23 09:44:00,1737625440,3000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,22265,True,False,declined,drop_flow_cashout


In [1008]:
amt_hand1.balance, amt_hand1.batch_txns, amt_hand1.declined_txns

(np.float64(25300.0), 5, 5)

**Отклоненное снятие частями**  
`BehavHand.online=False, receive=False, declined=True, BehavHand.in_chunks=True`

In [1009]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

behav_hand1.scen = "atm+transfer"
behav_hand1.in_chunks = True

receive_txn9 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns9 = [receive_txn9]
i = 0
while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out9 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=False, receive=False)
    all_txns9.append(part_out9)
    i += 1
    if i > 3:
        break
        
pd.DataFrame(all_txns9)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-13 20:13:00,1736799180,47000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-13 23:05:00,1736809500,23300.0,withdrawal,ATM,not applicable,False,NaN,Томск,56.484704,84.948174,not applicable,NaN,15206,True,False,declined,drop_flow_cashout
2,11794,2025-01-13 23:55:00,1736812500,17500.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,25211,True,False,declined,drop_flow_cashout
3,11794,2025-01-14 01:56:00,1736819760,11700.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,17059,True,False,declined,drop_flow_cashout
4,11794,2025-01-14 04:39:00,1736829540,5900.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,24922,True,False,declined,drop_flow_cashout


**НЕ отклоненный перевод другому дропу**  
Дропов >= `AccHand.min_drops`

In [784]:
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


6

In [785]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn10 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns10 = [receive_txn10]


behav_hand1.guide_scenario()
part_out10 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=True, receive=False)
all_txns10.append(part_out10)
all_df10 = pd.DataFrame(all_txns10)
all_df10

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-24 02:54:00,1737687240,36500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-24 04:11:00,1737691860,36500.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,14438,False,False,approved,not applicable


In [786]:
target_acc10 = all_df10.loc[1, "account"]
acc_hand1.accounts.query("account_id == @target_acc10")

,client_id,account_id,is_drop
4438,4695,14438,True


**НЕ отклоненный перевод другому дропу**  
Дропов < `AccHand.min_drops`

In [798]:
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops - 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


5

In [799]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn11 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns11 = [receive_txn11]


behav_hand1.guide_scenario()
part_out11 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=True, receive=False)
all_txns11.append(part_out11)
all_df11 = pd.DataFrame(all_txns11)
all_df11

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-21 07:10:00,1737443400,13700.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-21 09:57:00,1737453420,13700.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,15531,False,False,approved,not applicable


In [790]:
target_acc11 = all_df11.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc11]

2403    17772
Name: account_id, dtype: int64

**НЕ отклоненный перевод другому дропу**  
Дропов НЕТ

In [792]:
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


0

In [793]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn12 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns12 = [receive_txn12]

behav_hand1.guide_scenario()
part_out12 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=True, receive=False)
all_txns12.append(part_out12)
all_df12 = pd.DataFrame(all_txns12)
all_df12

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-01 21:59:00,1735768740,10000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-01 23:12:00,1735773120,10000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,25209,False,False,approved,not applicable


In [800]:
target_acc12 = all_df12.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc12]

9840    25209
Name: account_id, dtype: int64

**Отклоненный перевод другому дропу**  
Дропов >= AccHand.min_drops

In [1010]:
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops + 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


7

In [1012]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn13 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns13 = [receive_txn13]

behav_hand1.guide_scenario()
part_out13 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=True, receive=False)
all_txns13.append(part_out13)
all_df13 = pd.DataFrame(all_txns13)
all_df13

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-06 05:21:00,1736140860,37300.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-06 08:11:00,1736151060,37300.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,15034,True,False,declined,drop_flow_cashout


In [803]:
target_acc13 = all_df13.loc[1, "account"]
acc_hand1.accounts.query("account_id == @target_acc13")

,client_id,account_id,is_drop
1307,1377,11307,True


**Отклоненный перевод другому дропу**  
Дропов < AccHand.min_drops

In [1013]:
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops - 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


5

In [1014]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn14 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns14 = [receive_txn14]

behav_hand1.guide_scenario()
part_out14 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=True, receive=False)
all_txns14.append(part_out14)
all_df14 = pd.DataFrame(all_txns14)
all_df14

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-05 04:23:00,1736050980,78100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-05 07:03:00,1736060580,78100.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,22098,True,False,declined,drop_flow_cashout


In [806]:
target_acc14 = all_df14.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc14]

5542    20911
Name: account_id, dtype: int64

**Отклоненный перевод другому дропу**  
Дропов НЕТ

In [807]:
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


0

In [808]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn15 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns15 = [receive_txn15]

behav_hand1.guide_scenario()
part_out15 = cr_drop_txn1.trf_or_atm(declined=True, to_drop=True, receive=False)
all_txns15.append(part_out15)
all_df15 = pd.DataFrame(all_txns15)
all_df15

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-13 00:09:00,1736726940,84200.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-01-13 02:04:00,1736733840,84200.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,17583,True,False,declined,drop_flow_cashout


In [809]:
target_acc15 = all_df15.loc[1, "account"]
acc_hand1.outer_accounts.loc[acc_hand1.outer_accounts == target_acc15]

2214    17583
Name: account_id, dtype: int64

**Доля переводов дропам**  
Дропов `>=` AccHand.min_drops. Проверка в цикле

In [811]:
min_drops = dist_configs.to_drops["min_drops"]
print(min_drops)
acc_hand1.accounts["is_drop"] = False
acc_hand1.get_account(own=True)

accs_samp = acc_hand1.accounts.query("client_id != @acc_hand1.client_id").client_id.sample(n=min_drops + 1)
acc_hand1.accounts.loc[acc_hand1.accounts.client_id.isin(accs_samp), "is_drop"] = True
acc_hand1.accounts.query("client_id != @own_id and is_drop == True").shape[0]

6


7

In [823]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)
acc_hand1.reset_cache()

behav_hand1.scen = "split_transfer"
all_txns16

i = 0
while i < 300:
    receive_txn16 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
    all_txns16.append(receive_txn16)
    

    while amt_hand1.balance > 0:
        behav_hand1.in_chunks_val()
        behav_hand1.guide_scenario()
        to_drop16 = behav_hand1.to_drop
        part_out16 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=to_drop16, receive=False)
        all_txns16.append(part_out16)
    i += 1
    
all_df16 = pd.DataFrame(all_txns16)
print(f"{all_df16.shape[0]} rows")
all_df16.head()

2966 rows


,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-03-25 11:17:00,1742901420,62900.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206,False,False,approved,not applicable
1,11794,2025-03-26 05:59:00,1742968740,24000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,19214,False,False,approved,not applicable
2,11794,2025-03-26 07:50:00,1742975400,14000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,12633,False,False,approved,not applicable
3,11794,2025-03-26 09:37:00,1742981820,11000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,19873,False,False,approved,not applicable
4,11794,2025-03-26 10:41:00,1742985660,11000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,15832,False,False,approved,not applicable


In [828]:
own_id16 = acc_hand1.client_id
drop_accounts16 = acc_hand1.accounts.query("client_id != @own_id16 and is_drop == True")
df16_merged = all_df16.merge(drop_accounts16, how="outer", left_on="account", right_on="account_id").sort_values("unix_time")
df16_no_rcvd = df16_merged.query("type != 'inbound'")
df16_no_rcvd.head()

,client_id_x,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule,client_id_y,account_id,is_drop
2175,11794,2025-01-07 12:37:00,1736253420,13000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,21592,False,False,approved,not applicable,NaN,NaN,NaN
2180,11794,2025-01-07 14:50:00,1736261400,8900.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,21600,False,False,approved,not applicable,NaN,NaN,NaN
1143,11794,2025-01-08 09:55:00,1736330100,6000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9366.0,16415,False,False,approved,not applicable,NaN,NaN,NaN
168,11794,2025-01-08 10:37:00,1736332620,4000.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,13517,False,False,approved,not applicable,3713.0,13517.0,True
1774,11794,2025-01-08 12:22:00,1736338920,1600.0,outbound,transfer,not applicable,True,NaN,Томск,56.484704,84.948174,2.60.20.87,9367.0,19656,False,False,approved,not applicable,NaN,NaN,NaN


In [829]:
df16_no_rcvd.is_drop.value_counts(normalize=True, dropna=False)

is_drop
NaN     0.898013
True    0.101987
Name: proportion, dtype: float64

**Отклоненный  исходящий перевод частями и заодно набросок полного цикла дропа распределителя**

In [1794]:
# Объявление новых объектов классов для тестов.
drop_txn_part_data = FraudTransPartialData(merchants_df=pd.DataFrame(), client_info=clients_with_geo.loc[0], \
                                        online_merchant_ids=pd.DataFrame(), fraud_ips=fraud_ips, used_ips=pd.Series(), \
                                         fraud_devices=fraud_devices,  used_devices=pd.Series(), \
                                        client_devices=client_devices)


drop_client_test2 = DropClient(accounts=accounts, account=1, outer_accounts=outer_accounts)

create_drop_txn_tst = CreateDropTxn(timestamps=drop_stamps, trans_partial_data=drop_txn_part_data, drop_client_cls=drop_client_test2, \
                                      in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, attempts=0)
create_drop_txn_tst.reset_cache(only_counters=False)
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, drop_client_test2.batch_txns, create_drop_txn_tst.limit_reached()

(0, 0, 0, False)

In [ ]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
all_txns5 = []
declined=False
create_drop_txn_tst.attempts_after_decline(min=0, max=1)
drop_client_test2.chunk_size = 5000
drop_client_test2.batch_txns = 1

i = 1

while True:
    receive_txn8 = create_drop_txn_tst.single_operation(online=True, declined=declined, in_chunks=False, receive=True)
    all_txns5.append(receive_txn8)
    if declined:
        break
        
    while drop_client_test2.balance > 0:
        part_out = create_drop_txn_tst.cash_flow_action(online=True, receive=False, declined=declined, in_chunks=True)
        all_txns5.append(declined)
        print(f"iter {i}")
        create_drop_txn_tst.deduct_attempts(declined=declined, receive=False) 
        if create_drop_txn_tst.stop_after_decline(declined=declined):
            break
           
        declined = create_drop_txn_tst.limit_reached()
        i += 1
        
    # pd.DataFrame(all_txns5)

Ниже нерабочий код т.к. это прикидки с криптой

In [ ]:
# create_drop_txn_tst.reset_cache(only_counters=False)
# drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
# all_txns5 = []
# declined=False
# crypto_rate = 0.05
# create_drop_txn_tst.attempts_after_decline(min=0, max=1)
# drop_client_test2.chunk_size = 5000
# drop_client_test2.batch_txns = 1

# i = 1

# while True:
#     receive_txn8 = create_drop_txn_tst.single_operation(online=True, declined=declined, in_chunks=False, receive=True)
#     all_txns5.append(receive_txn8)
#     if declined:
#         break
        
#     while drop_client_test2.balance > 0:
#         if np.random.uniform(0, 1) < crypto_rate:
#             # использовать метод FraudTransPartialData.original_purchase внутри
#             part_out = create_drop_txn_tst.purchase(online=True, declined=declined, in_chunks=True, channel="crypto_exchange")
#         else:
#             part_out = create_drop_txn_tst.cash_flow_action(online=True, receive=False, declined=declined, in_chunks=True)
#         all_txns5.append(part_out)
#         print(f"iter {i}")
#         create_drop_txn_tst.deduct_attempts(declined=declined, receive=False) 
#         if create_drop_txn_tst.stop_after_decline(declined=declined):
#             break
           
#         declined = create_drop_txn_tst.limit_reached()
#         i += 1
        
#     # pd.DataFrame(all_txns5)

## **Тест `CreateTxn.purchase`**

### **`dist=True` дроп распределитель**

**НЕ отклоненная покупка крипты целиком**  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=False`

In [838]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)
acc_hand1.reset_cache()

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

receive_txn17 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns17 = [receive_txn17]

behav_hand1.guide_scenario()
whole_out17 = cr_drop_txn1.purchase(declined=False, dist=True)
all_txns17.append(whole_out17)
all_df17 = pd.DataFrame(all_txns17)
all_df17

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-28 08:04:00,1738051440,36000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-28 10:31:00,1738060260,36000.0,purchase,crypto_exchange,balance_top_up,True,6850.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,False,False,approved,not applicable


In [837]:
amt_hand1.batch_txns, time_hand1.in_txns, time_hand1.out_txns

(1, 1, 1)

**НЕ отклоненная покупка крипты частями**  
`BehavHand.online=True, receive=False, declined=False, BehavHand.in_chunks=True, BehavHand.scen = "split_transfer"`

In [862]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)

all_txns18 = []

receive_txn18 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns18.append(receive_txn18)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks_val()


while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out18 = cr_drop_txn1.purchase(declined=False, dist=True)
    all_txns18.append(part_out18)
    
all_df18 = pd.DataFrame(all_txns18)

In [863]:
all_df18

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-09 01:29:00,1736386140,67800.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-09 02:06:00,1736388360,12000.0,purchase,crypto_exchange,balance_top_up,True,6894.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,False,False,approved,not applicable
2,11794,2025-01-09 02:59:00,1736391540,33000.0,purchase,crypto_exchange,balance_top_up,True,6894.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,False,False,approved,not applicable
3,11794,2025-01-09 05:57:00,1736402220,13000.0,purchase,crypto_exchange,balance_top_up,True,6894.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,False,False,approved,not applicable
4,11794,2025-01-09 07:14:00,1736406840,9800.0,purchase,crypto_exchange,balance_top_up,True,6894.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,False,False,approved,not applicable


**Отклоненная покупка крипты целиком**  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=False`

In [867]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1)

all_txns19 = []

receive_txn19 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns19.append(receive_txn19)

behav_hand1.scen = "transfer"
behav_hand1.in_chunks_val()

i = 0
while amt_hand1.balance > 0 and i < 3:
    behav_hand1.guide_scenario()
    part_out19 = cr_drop_txn1.purchase(declined=True, dist=True)
    all_txns19.append(part_out19)
    i += 1
    
all_df19 = pd.DataFrame(all_txns19)
all_df19

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-01 15:24:00,1735745040,27600.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-01 17:12:00,1735751520,27600.0,purchase,crypto_exchange,balance_top_up,True,6858.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_flow_cashout
2,11794,2025-01-01 19:44:00,1735760640,27600.0,purchase,crypto_exchange,balance_top_up,True,6858.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_flow_cashout
3,11794,2025-01-01 20:50:00,1735764600,27600.0,purchase,crypto_exchange,balance_top_up,True,6858.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_flow_cashout


**Отклоненная  покупка крипты частями**  
`BehavHand.online=True, receive=False, declined=True, BehavHand.in_chunks=True`

In [869]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns20 = []

receive_txn20 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns20.append(receive_txn20)

behav_hand1.scen = "split_transfer"
behav_hand1.in_chunks_val()

i = 0
while amt_hand1.balance > 0 and i < 4:
    behav_hand1.guide_scenario()
    part_out20 = cr_drop_txn1.purchase(declined=True, dist=True)
    all_txns20.append(part_out20)
    i += 1
    
all_df20 = pd.DataFrame(all_txns20)
all_df20

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-30 15:52:00,1738252320,30500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-30 16:46:00,1738255560,24000.0,purchase,crypto_exchange,balance_top_up,True,6887.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,True,False,declined,drop_flow_cashout
2,11794,2025-01-30 18:41:00,1738262460,18000.0,purchase,crypto_exchange,balance_top_up,True,6887.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,True,False,declined,drop_flow_cashout
3,11794,2025-01-30 21:00:00,1738270800,14000.0,purchase,crypto_exchange,balance_top_up,True,6887.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,True,False,declined,drop_flow_cashout
4,11794,2025-01-30 23:13:00,1738278780,15000.0,purchase,crypto_exchange,balance_top_up,True,6887.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,True,False,declined,drop_flow_cashout


In [870]:
amt_hand1.balance

np.float64(30500.0)

### **`dist=False` дроп покупатель**

**НЕ отклоненная покупка целиком**  
`BehavHand.online=True, declined=False, BehavHand.in_chunks=False`

In [900]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns21 = []

receive_txn21 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns21.append(receive_txn21)

behav_hand1.online = True
behav_hand1.in_chunks = False

while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out21 = cr_drop_txn1.purchase(declined=False, dist=False)
    all_txns21.append(part_out21)
    
all_df21 = pd.DataFrame(all_txns21)
all_df21

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-11 17:34:00,1736616840,38600.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-11 18:50:00,1736621400,38600.0,purchase,ecom,shopping_net,True,6971.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,False,False,approved,not applicable


**НЕ отклоненная покупка частями**  
`BehavHand.online=True, declined=False, BehavHand.in_chunks=True, BehavHand.scen = "split_transfer"`

In [901]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns22 = []

receive_txn22 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns22.append(receive_txn22)

behav_hand1.online = True
behav_hand1.in_chunks = True

while amt_hand1.balance > 0:
    behav_hand1.guide_scenario()
    part_out22 = cr_drop_txn1.purchase(declined=False, dist=False)
    all_txns22.append(part_out22)
    
all_df22 = pd.DataFrame(all_txns22)
all_df22

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-10 03:25:00,1736479500,38600.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-10 05:32:00,1736487120,25000.0,purchase,ecom,misc_net,True,6791.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,False,False,approved,not applicable
2,11794,2025-01-10 06:27:00,1736490420,8000.0,purchase,ecom,misc_net,True,6960.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,False,False,approved,not applicable
3,11794,2025-01-10 07:06:00,1736492760,5600.0,purchase,ecom,shopping_net,True,6908.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,False,False,approved,not applicable


**Отклоненная покупка целиком**  
`BehavHand.online=True, declined=True, BehavHand.in_chunks=False`

In [1030]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns23 = []

receive_txn23 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns23.append(receive_txn23)

behav_hand1.online = True
behav_hand1.in_chunks = False

i = 0
while amt_hand1.balance > 0 and i < 4:
    part_out23 = cr_drop_txn1.purchase(declined=True, dist=False)
    all_txns23.append(part_out23)
    i += 1
    
all_df23 = pd.DataFrame(all_txns23)
all_df23

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-19 09:12:00,1737277920,48100.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-19 10:56:00,1737284160,48100.0,purchase,ecom,misc_net,True,6846.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_purchaser
2,11794,2025-01-19 12:11:00,1737288660,36100.0,purchase,ecom,shopping_net,True,6964.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_purchaser
3,11794,2025-01-19 12:55:00,1737291300,24100.0,purchase,ecom,shopping_net,True,6884.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_purchaser
4,11794,2025-01-19 14:52:00,1737298320,12100.0,purchase,ecom,shopping_net,True,6806.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_purchaser


In [1032]:
cum_sub_df = pd.DataFrame()
cum_sub_df["amt"] = np.arange(0, 5, step=1)
cum_sub_df["amt"] = - (48100 * 0.25 // 100 * 100)
cum_sub_df.loc[0, "amt"] = 48100
cum_sub_df.amt.cumsum()

0    48100.0
1    36100.0
2    24100.0
3    12100.0
4      100.0
Name: amt, dtype: float64

**Отклоненная  покупка частями**  
`BehavHand.online=True, declined=True, BehavHand.in_chunks=True`

In [1033]:
reset_caches(cr_drop_txn1, behav_hand1, amt_hand1, time_hand1, part_data1)

all_txns24 = []

receive_txn24 = cr_drop_txn1.trf_or_atm(declined=False, to_drop=False, receive=True)
all_txns24.append(receive_txn24)

behav_hand1.online = True
behav_hand1.in_chunks = True

i = 0
while amt_hand1.balance > 0 and i < 4:
    part_out24 = cr_drop_txn1.purchase(declined=True, dist=False)
    all_txns24.append(part_out24)
    i += 1
    
all_df24 = pd.DataFrame(all_txns24)
all_df24

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,11794,2025-01-15 10:18:00,1736936280,49500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,NaN,15206.0,False,False,approved,not applicable
1,11794,2025-01-15 11:25:00,1736940300,23000.0,purchase,ecom,shopping_net,True,6921.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_purchaser
2,11794,2025-01-15 13:22:00,1736947320,17300.0,purchase,ecom,travel_net,True,6868.0,Томск,56.484704,84.948174,2.60.20.87,9367.0,NaN,True,False,declined,drop_purchaser
3,11794,2025-01-15 15:20:00,1736954400,11600.0,purchase,ecom,shopping_net,True,6827.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,True,False,declined,drop_purchaser
4,11794,2025-01-15 17:20:00,1736961600,5900.0,purchase,ecom,shopping_net,True,6933.0,Томск,56.484704,84.948174,2.60.20.87,9366.0,NaN,True,False,declined,drop_purchaser


In [1034]:
cum_sub_df = pd.DataFrame()
cum_sub_df["amt"] = np.arange(0, 5, step=1)
cum_sub_df["amt"] = - (23000.0 * 0.25 // 100 * 100)
cum_sub_df.loc[0, "amt"] = 23000.0
cum_sub_df.amt.cumsum()

0    23000.0
1    17300.0
2    11600.0
3     5900.0
4      200.0
Name: amt, dtype: float64

**`limit_reached`**

In [1039]:
cr_drop_txn1.reset_cache()
# cr_drop_txn1.in_txns = cr_drop_txn1.in_lim - 1
cr_drop_txn1.out_txns = cr_drop_txn1.out_lim - 1
cr_drop_txn1.limit_reached()

False

**Конец теста `CreateDropTxn`**

# Конец информативной части

# Старый код, функции и т.д.

## Класс `DropClient`
- метод get_account(self): Мб добавить вариант to_drop: bool. Перевод другому имеющемуся дропу. при этом. 
    - записываем счет дропа в свой счет. Атрибут account
    - записываем  счет дропа в атрибут drop_accounts
    - если надо перевести другому дропу, то фильтруем drop_accounts исключая свой
    - надо тогда клиентам приписать шестизначные счета. просто по порядку.
    - счета куда отправляют дропы это номера начинающиеся со счета последнего клиента + 1

In [ ]:
# class DropClient: 
#     """
#     Генератор сумм входящих/исходящих транзакций, сумм снятий, номеров счетов.
#     Управление балансом текущего дропа.
#     """
#     def __init__(self, accounts, account, outer_accounts, balance=0, batch_txns=0, chunk_size=0, used_accounts=pd.Series(name="account_id")):
#         """
#         accounts - pd.DataFrame. Счета клиентов банка.
#         account - int. Номер счета текущего дропа.
#         outer_accounts - pd.Series. Номера счетов для входящих и исходящих переводов в/из других банков.
#         balance - float. Текущий баланс дропа
#         batch_txns - int. Счетчик транзакций сделанных в рамках распределения полученной партии денег
#         chunk_size - int, float. Последний созданный размер части баланса для перевода по частям.
#         used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
#         """
#         self.accounts = accounts
#         self.account = account
#         self.outer_accounts = outer_accounts
#         self.balance = balance
#         self.batch_txns = batch_txns
#         self.chunk_size = chunk_size
#         self.used_accounts = used_accounts

#     def update_balance(self, amount, add=False, declined=False):
#         """
#         Увеличить/уменьшить баланс на указанную сумму
#         -------------------
#         amount - float, int.
#         add - bool. Прибавлять сумму или отнимать.
#         declined - bool. Отклонена ли транзакция или одобрена.
#         """
#         # Не обновлять баланс если транзакция отклонена.
#         if declined:
#             return
            
#         # Увеличить баланс   
#         if add:
#             self.balance += amount
#             return
            
#         # Уменьшить баланс    
#         self.balance -= amount

#     def receive(self, declined, low=5000, high=100000, mean=30000, std=20000, round=500):
#         """
#         Генерация суммы входящего перевода
#         --------------------------
#         declined - bool. Отклонена ли транзакция или одобрена
#         low - float. Минимальная сумма
#         high - float. Максимальная сумма
#         mean - float. Средняя сумма
#         std - float. Стандартное отклонение
#         round - int. Округление целой части. По умолчанию 500. Значит числа будут либо с 500 либо с 000 на конце
#                      При условии что round не больше low и high. Чтобы отменить округление, нужно выставить 1
#         """

#         # Генерация суммы. Округление целой части при необходимости
#         amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // round * round
        
#         # Обновляем баланс если транзакция не отклонена
#         self.update_balance(amount=amount, add=True, declined=declined)
        
#         return amount

#     def get_chunk_size(self, online=False, atm_min=10000, atm_share=0.5, round=500, rand_rate=0.9, start=0, stop=0, step=0):
#         """
#         Вернуть случайный размер суммы перевода для перевода по частям
#         либо вернуть долю от баланса для снятия/перевода по частям.
#         -------------------------------
#         online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля atm_share от баланса, но не меньше atm_min
#         atm_min - int, float. Минимальная сумма снятия дропом в банкомате.
#         atm_share - float. Доля от баланса если снятие через банкомат.
#         round - int. Округление целой части. По умолчанию 500. 
#                      Значит суммы будут округлены до тысяч или пяти сотен
#         rand_rate - float. От 0 до 1. Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
#                            Но возможны случайные совпадаения с предыдущим размером чанка.
#                            Доля случайных размеров подряд будет:
#                            p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
#                            p(rr) - rand_rate
#                            p(rr) - (p(rr) * p(c)). Например p(rr) = 0.9, 5 вариантов размеров чанка - p(c) = 0.20
#                            0.9 - (0.9 * 0.2) = 0.72
#                            В около 72% случаев размеры чанков не будут подряд одинаковыми 
#         start - int. Минимальный размер. Прописываем если генерация не через share.
#                      То же самое для stop и step
#         stop - int. Максимальный размер - не входит в возможный выбор.
#                     Максимальное генерируемое значение равно stop - step
#         step - int. Шаг размеров.
#         --------------------
#         Возвращает np.int64
#         Результат кэшируется в self.chunk_size
#         """
#         # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
#         # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
#         if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
#             return self.chunk_size

#         # Если перевод
#         if online:
#             sampling_array = np.arange(start, stop, step)
#             self.chunk_size = np.random.choice(sampling_array)
#             return self.chunk_size
            
#         # Если снятие    
#         self.chunk_size = max(atm_min, self.balance * atm_share // round * round)
#         return self.chunk_size
            
        
#     def one_operation(self, amount=0, declined=False, in_chunks=False):
#         """
#         Генерация суммы операции дропа.
#         ---------
#         amount - float, int. Сумма перевода если перевод по частям - in_chunks == True
#         declined - bool. Отклонена ли транзакция или одобрена
#         in_chunks - bool. Перевод по частям или целиком. Если False, то просто пробуем перевести все с баланса
#                           При True нужно указать amount.
#         """
#         if in_chunks and amount <= 0:
#             raise ValueError(f"""If in_chunks is True, then amount must be greater than 0.
# Passed amount: {amount}""")

#         # Если перевод не по частям. Пробуем перевестит все с баланса. 
#         if not in_chunks:
#             amount = self.balance
#             self.update_balance(amount=self.balance, add=False, declined=declined)
#             # Прибавляем счетчик транзакции для текущей партии денег
#             self.batch_txns += 1
#             return amount

#         # Иначе считаем сколько частей исходя из размера одной части
#         chunks = self.balance // amount

#         # Если целое число частей больше 0. Пробуем перевести одну часть
#         if chunks > 0:
#             self.update_balance(amount=amount, add=False, declined=declined)
#             self.batch_txns += 1
#             return amount

#         # Если баланс меньше одной части. Пробуем перевести то что осталось
#         rest = self.balance
#         self.update_balance(amount=rest, add=False, declined=declined)
#         self.batch_txns += 1
#         return rest

#     def get_account(self, to_drop):
#         """
#         Номер счета входящего/исходящего перевода
#         to_drop - bool. Перевод другому дропу в нашем банке или нет.
#         """
#         # Фильтруем accounts исключая свой счет и выбирая дропов. Для случая если to_drop
#         drop_accounts = self.accounts.loc[(self.accounts.account_id != self.account) & (self.accounts.is_drop == True)]

#         # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
#         if to_drop and not drop_accounts.empty: 
#             account = drop_accounts.account_id.sample(1).iat[0]
#             # Добавляем этот счет в использованные как последнюю запись в серии
#             self.used_accounts.loc[self.used_accounts.shape[0]] = account
#             return account

#         # Если отправляем/получаем из другого банка.  
#         # Семплируем номер внешнего счета который еще не использовался
#         account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
#         # Добавляем этот счет в использованные как последнюю запись в серии
#         self.used_accounts.loc[self.used_accounts.shape[0]] = account
        
#         return account

#     def reset_cache(self, balance=True, used_accounts=True, chunk_size=True, batch_txns=True):
#         """
#         Сброс кэшированных значений
#         По умолчанию сбрасывается всё. Если что-то надо оставить, то надо выставить False
#         для этого
#         -----------------
#         balance - bool
#         used_accounts - bool
#         chunk_size - bool
#         batch_txns - bool
#         """
#         if balance:
#             self.balance = 0
#         if used_accounts:
#             self.used_accounts = pd.Series(name="account_id")
#         if chunk_size:
#             self.chunk_size = 0
#         if batch_txns:
#             self.batch_txns = 0